In [1]:
import pyspark
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


24/02/25 18:32:58 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/02/25 18:32:59 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [2]:

df_green = spark.read.parquet('data/pq/green/*/*')

In [3]:
df_green.registerTempTable('green')

/home/barbaradekeyser/spark/spark-3.3.2-bin-hadoop3/python/pyspark/sql/dataframe.py:229: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.
  warnings.warn("Deprecated in 2.0, use createOrReplaceTempView instead.", FutureWarning)


In [5]:
df_green_revenue = spark.sql("""
SELECT 
    date_trunc('hour', lpep_pickup_datetime) AS hour, 
    PULocationID AS zone,
    SUM(total_amount) AS amount,
    COUNT(1) AS number_records
FROM
    green
WHERE
    lpep_pickup_datetime >= '2020-01-01 00:00:00'
GROUP BY
    1, 2
""")
# only select 2020

In [6]:

df_green_revenue \
    .repartition(20) \
    .write.parquet('data/report/revenue/green', mode='overwrite')

In [7]:
df_yellow = spark.read.parquet('data/pq/yellow/*/*')
df_yellow.registerTempTable('yellow')

In [8]:
df_yellow_revenue = spark.sql("""
SELECT 
    date_trunc('hour', tpep_pickup_datetime) AS hour, 
    PULocationID AS zone,

    SUM(total_amount) AS amount,
    COUNT(1) AS number_records
FROM
    yellow
WHERE
    tpep_pickup_datetime >= '2020-01-01 00:00:00'
GROUP BY
    1, 2
""")

In [9]:

df_yellow_revenue \
    .repartition(20) \
    .write.parquet('data/report/revenue/yellow', mode='overwrite')

In [10]:
df_green_revenue = spark.read.parquet('data/report/revenue/green')
df_yellow_revenue = spark.read.parquet('data/report/revenue/yellow')

In [19]:
df_green_revenue_tmp.show()

+-------------------+----+------------------+--------------------+
|               hour|zone|      green_amount|green_number_records|
+-------------------+----+------------------+--------------------+
|2020-06-23 11:00:00|  14|             71.28|                   2|
|2021-01-20 07:00:00| 131|              52.9|                   1|
|2020-12-16 14:00:00|  77|            122.53|                   3|
|2020-11-25 11:00:00| 188|             53.14|                   2|
|2020-09-22 07:00:00| 188|             76.15|                   3|
|2021-05-20 04:00:00|  62|             37.67|                   1|
|2020-04-09 04:00:00|  14|              20.9|                   1|
|2021-03-26 10:00:00|  74|317.35000000000014|                  22|
|2020-11-14 18:00:00|  75|            327.97|                  17|
|2021-07-16 13:00:00|  72|             26.36|                   1|
|2020-11-25 11:00:00| 222|             33.62|                   1|
|2020-10-03 19:00:00| 198|             40.86|                 

In [11]:
df_green_revenue_tmp = df_green_revenue \
    .withColumnRenamed('amount', 'green_amount') \
    .withColumnRenamed('number_records', 'green_number_records')

df_yellow_revenue_tmp = df_yellow_revenue \
    .withColumnRenamed('amount', 'yellow_amount') \
    .withColumnRenamed('number_records', 'yellow_number_records')

In [12]:
df_join = df_green_revenue_tmp.join(df_yellow_revenue_tmp, on=['hour', 'zone'], how='outer')

In [13]:
df_join.write.parquet('data/report/revenue/total', mode='overwrite')

In [14]:
df_join = spark.read.parquet('data/report/revenue/total')

In [15]:
df_join

DataFrame[hour: timestamp, zone: int, green_amount: double, green_number_records: bigint, yellow_amount: double, yellow_number_records: bigint]

In [16]:
df_zones = spark.read.parquet('zones/')

In [17]:
df_result = df_join.join(df_zones, df_join.zone == df_zones.LocationID)

In [20]:
df_result.show()

+-------------------+----+------------------+--------------------+------------------+---------------------+----------+---------+--------------------+------------+
|               hour|zone|      green_amount|green_number_records|     yellow_amount|yellow_number_records|LocationID|  Borough|                Zone|service_zone|
+-------------------+----+------------------+--------------------+------------------+---------------------+----------+---------+--------------------+------------+
|2020-01-01 00:00:00|  14|              null|                null|               8.8|                    1|        14| Brooklyn|           Bay Ridge|   Boro Zone|
|2020-01-01 00:00:00|  15|              null|                null|             34.09|                    1|        15|   Queens|Bay Terrace/Fort ...|   Boro Zone|
|2020-01-01 00:00:00|  17|195.03000000000003|                   9|220.20999999999998|                    8|        17| Brooklyn|             Bedford|   Boro Zone|
|2020-01-01 00:00:00| 

In [18]:
df_result.drop('LocationID', 'zone').write.parquet('tmp/revenue-zones')

In [21]:
spark.stop()